# Actualizar datos

## Descripción
El proceso para actualizar los datos diariamente consiste en los siguientes pasos:

1. Descargar archivos.
2. Procesar el `pdf` y convertir en `tsv`.
3. Agregar información a la tabla de relación fecha-archivo.
4. Actualizar tabla general.
5. Revisión de inconsistencias.

## Código

In [90]:
import requests
from bs4 import BeautifulSoup
import camelot
import pandas as pd
import datetime
import numpy as np

In [92]:
def extract_documents(url, outdir='.'):
    '''
    Descarga de documentos de la página de la Secretaría de Salud
    '''
    
    # obtener sitio
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # encotnrar seccion de documentos
    # 2020.03.25: Seccion de documentos bajo <div class="clearfix">
    documents = soup.find('div', attrs={'class': 'clearfix'})
            
    # links de los documentos
    file_urls = []
    
    for href in documents.find_all('a', href=True):
        file_urls.append(href['href'])
        
    # descargar
    for i in range(len(file_urls)):
        file_url = file_urls[i]
        file_name = file_urls[i].split('/')[-1]
        response = requests.get(f'https://www.gob.mx{file_url}')
        
        with open(f'{outdir}/{file_name}', 'wb') as f:
            f.write(response.content)
            print(f'Archivo: {file_name}... descargado.')

def parse_pdf(file, paginas='all', casos_nuevos=None):
    '''
    
    '''
    tables = camelot.read_pdf(file, pages=paginas, flavor='stream', row_tol=10)
    print('{} paginas'.format(tables.n))

    # juntar todas las tablas
    tables_list = []
    for i in range(0,tables.n):
        df_i = tables[i].df
        
        # buscar columnas con nan
        df_i= df_i.replace(r'^\s*$', np.nan, regex=True)
        df_i = df_i.replace('NA', np.nan)
        
        # remove na columns
        df_i = df_i.dropna(axis='columns', thresh=10)
        df_i = df_i.T.reset_index(drop=True).T
        
        # quitar filas de titulo
        # por si proceso una pagina vacia
        if len(df_i.columns) != 0:
            df_i[0] = df_i[0].fillna('notnum')
            df_i = df_i[df_i[0].apply(lambda x: x.isnumeric())]
        
            # argregar a lista
            tables_list.append(df_i)

    # juntar tabla unica
    df = pd.concat(tables_list)
    
    
    # renombrar columnas
    df.columns = ['num_caso', 'estado', 'sexo', 'edad', 'fecha_inicio_sintomas', 'id_rt-pcr', 'procedencia', 'fecha_llegada_mx']
    
    # actualizar index
    df.set_index('num_caso', inplace=True, verify_integrity=True)
    df.index.name = None
    
    # convertir a datetime format
    df['fecha_inicio_sintomas'] = pd.to_datetime(df['fecha_inicio_sintomas'], format='%d/%m/%Y')
    df['fecha_llegada_mx'] = pd.to_datetime(df['fecha_llegada_mx'], format='%d/%m/%Y')
    
    # limpiar nombres de estados
    df['estado'] = df['estado'].str.title()
    df['estado'] = df['estado'].str.replace('Ciudad De México', 'Ciudad de México')
    df['estado'] = df['estado'].str.replace('"Estados \nUnidos"', 'Estados Unidos')
    
    # CASOS NUEVOS
    
    if casos_nuevos:
        # interpretar string de casos nuevos
        # by: https://stackoverflow.com/users/190597/unutbu
        # in: https://stackoverflow.com/questions/4726168/parsing-command-line-input-for-numbers
        result = set()

        for part in casos_nuevos.split(','):
            x = part.split('-')
            result.update(range(int(x[0]), int(x[-1]) + 1))

        lista_casos_nuevos = sorted(result)

        # generar lista con anotaciones de casos nuevos
        lista_nuevos = []

        for i in df.index:
                if int(i) in lista_casos_nuevos:
                    lista_nuevos.append(True)
                else:
                    lista_nuevos.append(False)

        df['casos_nuevos'] = lista_nuevos
    
    else:
        print('No se agregara información sobre casos nuevos. (Celdas en azul.)')

    return df

def save_tsv(pdf_name, save_name, celdas_azules):
    
    df = parse_pdf(f'{pdf_name}',
                   paginas='all',
                   casos_nuevos=celdas_azules)
    
    df.to_csv(f'{save_name}',
              sep='\t',
              index=True)

def create_dict(df_files):
    
    dict_df = {}
    
    for column, row in df_files.iterrows():
        df = pd.read_csv(row['archivo_tsv'],
                         sep='\t',
                         parse_dates=['fecha_inicio_sintomas', 'fecha_llegada_mx'],
                         index_col=0)

        dict_df[row['fecha']] = df
    
    return dict_df

def inconsistencias(dict_tablas):
    
    results = []
    
    ordered_dates = sorted(dict_tablas.keys())
    
    for date_1, date_2 in zip(ordered_dates, ordered_dates[1:]):

        df1 = dict_df[date_1]
        df2 = dict_df[date_2]

        # diferencia de longitud entre dos días
        len_diff = len(df2) - len(df1)

        # seleccionar solo casos nuevos en día 2
        nuevos = len(df2[df2['casos_nuevos'] == True])

        date_1_str = date_1.strftime('%d-%m-%Y')
        date_2_str = date_2.strftime('%d-%m-%Y')

        if len_diff == nuevos:
            print(f'{date_1_str} a {date_2_str}: ✓')

        else:
            print(f'{date_1_str} a {date_2_str}: inconsistente por {len_diff-nuevos}')


            # add dates
            df1['casos_nuevos'] = df1['casos_nuevos'].replace(True, date_1)
            df2['casos_nuevos'] = df2['casos_nuevos'].replace(True, date_2)

            # add pseudo index
            df1['pseudo_indice'] = date_1.strftime('%Y%m%d_') + df1.index.astype(str)
            df2['pseudo_indice'] = date_2.strftime('%Y%m%d_') + df2.index.astype(str)

            # check duplicates
            df_joint = pd.concat([df1, df2[df2['casos_nuevos'] == False]])


            df_joint['duplicado'] = df_joint.duplicated(subset=['estado', 'sexo', 'edad', 'fecha_inicio_sintomas', 'procedencia'], keep=False)

            df_inc = df_joint[df_joint['duplicado'] == False]
            df_inc.info = f'{date_1_str} a {date_2_str}: inconsistente por {len_diff-nuevos}'
            
            #df_inc.to_csv('../datos/tablas_procesadas/err_log/{}-{}.tsv'.format(date_1_str, date_2_str),
             #             sep='\t',
              #            index=None)
            
            results.append(df_inc)
            
    return results

def update(df_files, df_completa):
    
    dict_df = create_dict(df_files)
    
    # encontrar fecha mas reciente añadida a la tabla completa
    mas_reciente = sorted(df_completa['fecha_caso_nuevo'].to_list())[-1]
    # encontrar fechas en los archivos que no han sido añadidas a la tabla completa
    to_update = [date for date in sorted(df_files['fecha'].to_list()) if date > mas_reciente]
    
    if len(to_update) == 0:
        print('No hay nada que actualizar.')
        
    else:
        for date in to_update:
            # leer tabla
            df_update = dict_df[date]

            # add pseudo index
            df_update['pseudo_indice'] = date.strftime('%Y%m%d_') + df_update.index.astype(str)
            df_update = df_update[df_update['casos_nuevos'] != False]

            # agregar fechas a las nuevas
            df_update = df_update.copy()
            df_update['fecha_caso_nuevo'] = df_update['casos_nuevos'].replace(True, date)
            df_update = df_update.drop(columns=['casos_nuevos'])

            df_update = df_update[['pseudo_indice', 'estado', 'sexo', 'edad', 'fecha_inicio_sintomas', 
                     'id_rt-pcr', 'procedencia', 'fecha_llegada_mx', 'fecha_caso_nuevo']]

            # agregar a df anterior
            df_nuevo = pd.concat([df_completa, df_update])

        return df_nuevo

Leer tabla completa

In [93]:
df_completa = pd.read_csv('../datos/tablas_procesadas/tabla_completa.tsv',
                          sep='\t',
                          parse_dates=['fecha_inicio_sintomas', 'fecha_llegada_mx', 'fecha_caso_nuevo'])

Crear diccionario de fechas-tablas

In [94]:
dict_df = create_dict(df_files)

## Resultados

### 1. Descargar archivos

In [75]:
url = 'https://www.gob.mx/salud/documentos/coronavirus-covid-19-comunicado-tecnico-diario-238449'
extract_documents(url, outdir='../datos/ssalud_pdf')

Archivo: Comunicado_Tecnico_Diario_COVID-19_2020.03.29.pdf... descargado.
Archivo: Tabla_casos_sospechosos_COVID-19_2020.03.29.pdf... descargado.
Archivo: Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.29.pdf... descargado.


### 2. Procesar el `pdf` y convertir en `tsv`.

#### 2020-03-29

In [76]:
pdf = '../datos/ssalud_pdf/Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.29.pdf'
celdas_azules = '''769, 772, 776, 778, 781, 782, 785, 792, 794, 796, 798-803, 
                   805, 806, 808, 811, 813, 814, 821, 823, 824, 826-828, 830,
                   831, 835, 837, 844-847, 850, 852, 854, 856, 861, 867, 868,
                   870-873, 876-881, 885, 887-894, 896-898, 900-902, 904, 906,
                   907, 909, 910, 911, 912, 913, 915-919, 921, 924, 926-932, 939-993'''

save_name = '../datos/tablas_originales/20200329_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

19 paginas


In [36]:
print(f'Converted:')
print(f'{pdf} to')
print(f'{save_name}')

Converted:
../datos/ssalud_pdf/Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.29.pdf to
../datos/tablas_originales/20200329_positivos.tsv


### 3. Agregar información a la tabla de relación fecha-archivo.

In [67]:
df_info = pd.read_csv('../datos/tablas_procesadas/info_tablas.tsv',
                      sep='\t',
                      parse_dates=['fecha'])

In [50]:
df_info = df_info.append({'fecha': datetime.datetime(2020,3,29), 
                          'pdf_original': pdf,
                          'archivo_tsv': save_name}, ignore_index=True)

In [52]:
df_info.to_csv('../datos/tablas_procesadas/info_tablas.tsv',
               sep='\t',
               index=None)

In [60]:
print('Added:')

Added:


In [58]:
df_info.tail(1)

,fecha,pdf_original,archivo_tsv
13,2020-03-29,../datos/ssalud_pdf/Tabla_casos_positivos_COVI...,../datos/tablas_originales/20200329_positivos.tsv


### 4. Actualizar tabla

In [95]:
df_nuevo = update(df_info, df_completa)

In [100]:
df_nuevo

,pseudo_indice,estado,sexo,edad,fecha_inicio_sintomas,id_rt-pcr,procedencia,fecha_llegada_mx,fecha_caso_nuevo
0,20200316_1,Ciudad de México,M,35,2020-02-22,confirmado,Italia,2020-02-22,NaT
1,20200316_2,Sinaloa,M,41,2020-02-22,confirmado,Italia,2020-02-21,NaT
2,20200316_3,Ciudad de México,M,59,2020-02-23,confirmado,Italia,2020-02-22,NaT
3,20200316_4,Coahuila,F,20,2020-02-27,confirmado,Italia,2020-02-25,NaT
4,20200316_5,Chiapas,F,18,2020-02-25,confirmado,Italia,2020-02-25,NaT
...,...,...,...,...,...,...,...,...,...
989,20200329_989,México,F,31,2020-03-20,confirmado,Contacto,NaT,2020-03-29
990,20200329_990,Oaxaca,F,52,2020-03-27,confirmado,Contacto,NaT,2020-03-29
991,20200329_991,Oaxaca,M,44,2020-03-26,confirmado,Estados Unidos,NaT,2020-03-29
992,20200329_992,Baja California Sur,M,50,2020-03-21,confirmado,España,NaT,2020-03-29


In [101]:
diff = len(df_nuevo) - len(df_completa)
print(f'Se agregaron {diff} entradas.')

Se agregaron 145 entradas.


In [102]:
save_as = '../datos/tablas_procesadas/tabla_completa.tsv'
df_nuevo.to_csv(save_as,
                sep='\t',
                index=None)

print(f'Se actualizó tabla en: {save_as}')

Se actualizó tabla en: ../datos/tablas_procesadas/tabla_completa.tsv


### 5. Revisión de inconsistencias

In [84]:
tablas_inc = inconsistencias(dict_df)

16-03-2020 a 17-03-2020: ✓
17-03-2020 a 18-03-2020: ✓
18-03-2020 a 19-03-2020: ✓
19-03-2020 a 20-03-2020: inconsistente por -1
20-03-2020 a 21-03-2020: ✓
21-03-2020 a 22-03-2020: inconsistente por 4
22-03-2020 a 23-03-2020: ✓
23-03-2020 a 24-03-2020: ✓
24-03-2020 a 25-03-2020: ✓
25-03-2020 a 26-03-2020: inconsistente por -1
26-03-2020 a 27-03-2020: inconsistente por -1
27-03-2020 a 28-03-2020: inconsistente por -1


In [85]:
ultima = sorted(df_files['fecha'])[-1].strftime('%Y-%m-%d')
penultima = sorted(df_files['fecha'])[-2].strftime('%Y-%m-%d')

print('Comparación más reciente: {} a {}'.format(penultima, ultima))

Comparación más reciente: 2020-03-27 a 2020-03-28


In [13]:
tablas_inc[-1].to_csv('../datos/tablas_procesadas/err_log/{}_{}.tsv'.format(penultima, ultima),
                      sep='\t',
                      index=None)

In [14]:
tablas_inc[-1]

,estado,sexo,edad,fecha_inicio_sintomas,id_rt-pcr,procedencia,fecha_llegada_mx,casos_nuevos,pseudo_indice,duplicado
585,Chiapas,F,32,2020-03-20,confirmado,Estados Unidos,2020-07-03,2020-03-27 00:00:00,20200327_585,False
